In [1]:
import os 
import sys 

cur_path = os.path.abspath(".")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

from utils import remove_prefix, ddf 

load_dotenv()

True

In [4]:
sg = Subgrounds()
bs: Subgraph = sg.load_subgraph(os.environ['SUBGRAPH_URL'])

# Fertilizer Purchased by Humidity 

In [5]:
pool = bs.Query.metapoolOracles(first=10000, orderBy="season", orderDirection="asc")
df_pool_raw = sg.query_df(
    [
        pool.balanceA, 
        pool.balanceB, 
        pool.season, 
        pool.deltaB, 
    ],
    pagination_strategy=ShallowStrategy
)

In [20]:
df_pool = df_pool_raw.copy()
df_pool = remove_prefix(df_pool, "metapoolOracles_")
df_pool.balanceA /= 10**6
df_pool.balanceB /= 10**18
df_pool['diff'] = df_pool.balanceB - df_pool.balanceA
print(len(df_pool))
df_pool.tail()

129


,balanceA,balanceB,season,deltaB,diff
124,9.696624e+12,9961026314965.550781,6200,-224968582,264402707684.984375
125,9.762227e+12,10025226281979.396484,6201,94340,262999526805.75
126,9.826355e+12,10087949335089.861328,6202,-4846037446,261594064185.205078
127,9.891124e+12,10151290212800.416016,6203,-6019669737,260166294657.839844
128,9.955877e+12,10214646398975.939453,6204,-1673313940,258769497315.191406


In [117]:
nearest = alt.selection_single(
    nearest=True, encodings=["x"], on="mouseover", empty="none"
)
brush = alt.selection_interval(encodings=['x'])

base = alt.Chart(df_ftokens)
x = alt.X("humidity:Q")
y = alt.Y("fertilizer:Q")

# Histogram of fertilizer purchased by season 
histogram = base.mark_bar().encode(
    x=x, y=y
)

# Vertical line to indicate nearest histogram bin 
vline = base.mark_rule(color="grey").encode(
    x=x, size=alt.value(1)
).transform_filter(
    nearest
)

# Text label to indicate value nearest histogram bin 
text = histogram.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, y, alt.value(' '))
)

# Time axis which drives scale of histogram 
time_axis = base.mark_bar().encode(
    x=x, y=y
).add_selection(
    brush 
).resolve_scale(
    'independent'
).properties(height=50)

(
    alt.layer(
        histogram.add_selection(nearest), 
        text, 
        vline
    ).transform_filter(
        brush
    ).properties(height=250) 
    & time_axis 
) 

alt.VConcatChart(...)

# Chopping 

In [6]:
ferts = bs.Query.fertilizerBalances(first=10000)

df_ferts_raw = sg.query_df(
    [
        ferts.id, 
        ferts.amount, 
        ferts.farmer, 
        ferts.fertilizerToken, 
    ],
    pagination_strategy=ShallowStrategy
)
#.sort_values('podFills_transaction_timestamp').reset_index(drop=True)

In [7]:
df_ferts = df_ferts_raw.copy()
print(len(df_ferts))
print(df_ferts.fertilizerBalances_amount.sum())
print(df_ferts.fertilizerBalances_amount.sum())
print(df_ferts.columns)
df_ferts.head()

1047
17251956
17251956
Index(['fertilizerBalances_id', 'fertilizerBalances_amount',
       'fertilizerBalances_farmer_id', 'fertilizerBalances_farmer_silo_id',
       'fertilizerBalances_farmer_field_id', 'fertilizerBalances_farmer_sown',
       'fertilizerBalances_fertilizerToken_id',
       'fertilizerBalances_fertilizerToken_fertilizer_id',
       'fertilizerBalances_fertilizerToken_supply',
       'fertilizerBalances_fertilizerToken_humidity',
       'fertilizerBalances_fertilizerToken_endBpf',
       'fertilizerBalances_fertilizerToken_startBpf',
       'fertilizerBalances_fertilizerToken_season'],
      dtype='object')


,fertilizerBalances_id,fertilizerBalances_amount,fertilizerBalances_farmer_id,fertilizerBalances_farmer_silo_id,fertilizerBalances_farmer_field_id,fertilizerBalances_farmer_sown,fertilizerBalances_fertilizerToken_id,fertilizerBalances_fertilizerToken_fertilizer_id,fertilizerBalances_fertilizerToken_supply,fertilizerBalances_fertilizerToken_humidity,fertilizerBalances_fertilizerToken_endBpf,fertilizerBalances_fertilizerToken_startBpf,fertilizerBalances_fertilizerToken_season
0,3005557-0xe27bdf8c099934f425a1c604dfc9a82c3b3d...,283,0xe27bdf8c099934f425a1c604dfc9a82c3b3dd458,0xe27bdf8c099934f425a1c604dfc9a82c3b3dd458,None,True,3005557,0x402c84de2ce49af88f5e2ef3710ff89bfed36cb6,283,187.5,3005557,130557,6199
1,3015555-0x3800645f556ee583e20d6491c3a60e9c3274...,997,0x3800645f556ee583e20d6491c3a60e9c32744376,0x3800645f556ee583e20d6491c3a60e9c32744376,None,True,3015555,0x402c84de2ce49af88f5e2ef3710ff89bfed36cb6,997,188.5,3015555,130555,6197
2,3045431-0x507165ff0417126930d7f79163961de8ff19...,955,0x507165ff0417126930d7f79163961de8ff19c8b8,None,None,True,3045431,0x402c84de2ce49af88f5e2ef3710ff89bfed36cb6,955,191.5,3045431,130431,6191
3,3060408-0x8dbd580e34a9ae2f756f14251bff64c14d32...,564,0x8dbd580e34a9ae2f756f14251bff64c14d32124c,0x8dbd580e34a9ae2f756f14251bff64c14d32124c,None,False,3060408,0x402c84de2ce49af88f5e2ef3710ff89bfed36cb6,564,193.0,3060408,130408,6188
4,3095329-0x995d1e4e2807ef2a8d7614b607a89be09631...,451,0x995d1e4e2807ef2a8d7614b607a89be096313916,0x995d1e4e2807ef2a8d7614b607a89be096313916,None,True,3095329,0x402c84de2ce49af88f5e2ef3710ff89bfed36cb6,451,196.5,3095329,130329,6181


In [8]:
"""
fertilizer balances by season 
"""

df_ferts = df_ferts_raw.copy()
df_ferts = df_ferts.rename(columns={
    'fertilizerBalances_amount': 'amount',
    'fertilizerBalances_farmer_id': 'address', 
    'fertilizerBalances_fertilizerToken_humidity': 'humidity',
    'fertilizerBalances_fertilizerToken_season': 'season' # season fertilizer was minted in 
})
df_ferts = df_ferts[['amount', 'address', 'season', 'humidity']].sort_values("season")

# ddf(df_ferts)